In [38]:
import requests
import pandas as pd
import matplotlib
import os
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import numpy as np

In [6]:
# API Key to use Polygon.io
API_KEY = 'iSTiF04K1tV__pkyh2UvhqBPheoJhKaO'

# URL Strings to Pull Data
TICKERS_URL = 'https://api.polygon.io/v3/reference/tickers?active=true&sort=ticker&order=asc&limit={}&apiKey={}'
FINANCIAL_URL = 'https://api.polygon.io/v2/reference/financials/{}?limit={}&type={}&sort=calendarDate&apiKey={}'

In [7]:
# Get Ticker Data
limit = 100

session = requests.Session()
r = session.get(TICKERS_URL.format(limit, API_KEY))
data = r.json()
data

{'results': [{'ticker': 'A',
   'name': 'Agilent Technologies Inc.',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'XNYS',
   'type': 'CS',
   'active': True,
   'currency_name': 'usd',
   'cik': '0001090872',
   'composite_figi': 'BBG000BWQYZ5',
   'share_class_figi': 'BBG001SCTQY4',
   'last_updated_utc': '2021-10-17T00:00:00Z'},
  {'ticker': 'AA',
   'name': 'Alcoa Corporation',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'XNYS',
   'type': 'CS',
   'active': True,
   'currency_name': 'usd',
   'cik': '0001675149',
   'composite_figi': 'BBG00B3T3HD3',
   'share_class_figi': 'BBG00B3T3HF1',
   'last_updated_utc': '2021-10-17T00:00:00Z'},
  {'ticker': 'AAA',
   'name': 'AAF First Priority CLO Bond ETF',
   'market': 'stocks',
   'locale': 'us',
   'primary_exchange': 'ARCX',
   'type': 'ETF',
   'active': True,
   'currency_name': 'usd',
   'composite_figi': 'BBG00X5FSP48',
   'share_class_figi': 'BBG00X5FSPZ4',
   'last_updated_utc': '2021-10-1

In [110]:
# Get Financial Data From Polygon.io
def GET_FINANCIALS(TICKER_F,LIMIT):
    r_F = session.get(FINANCIAL_URL.format(TICKER_F,LIMIT,'YA',API_KEY))
    data_F = r_F.json()
    fin_df = pd.DataFrame(data_F['results'])
    return(fin_df)

In [111]:
# Growth Rate
def calc_GR(P0, P1, YEARS):
    rate = (P1/P0)**(1/YEARS) - 1
    return rate

In [112]:
# Set Ticker and Get Data
fin_df = GET_FINANCIALS('PCP', 100)
n = len(fin_df)

# ROIC
ROIC = fin_df[['calendarDate','returnOnInvestedCapital']]
if (n > 10):
    ROIC_arr = ROIC.iloc[[n-11,n-6,n-4,n-1]]
    ROIC_df = ROIC_arr.reset_index(drop=True)

# Set Growth Metric Indexes
growthMetrics = ['Sales Growth','EPS Growth','Equity Growth','Free Cash Growth']
growthIndexes = ['revenues','earningsPerBasicShare','bookValuePerShare','freeCashFlow']
m = len(growthMetrics)

# Set Years to Calculate Growth Rates
GR_YEARS = np.array([10, 5, 3, 1])
k = len(GR_YEARS)

GR = np.zeros((k,m))
for j in range(0, m):
    Index = growthIndexes[j]
    for i in range(0, k):
        years = GR_YEARS[i]
        p1 = fin_df.loc[n - 1, Index]
        p0 = fin_df.loc[n - 1 - years, Index]
        GR[i,j] = calc_GR(p0, p1, years)

growthRates = pd.DataFrame(GR, columns=growthMetrics)
bigFive = pd.concat([ROIC_df, growthRates], axis=1)
bigFive

<ipython-input-111-13ede3585c6c>:3: RuntimeWarning: invalid value encountered in double_scalars
  rate = (P1/P0)**(1/YEARS) - 1


,calendarDate,returnOnInvestedCapital,Sales Growth,EPS Growth,Equity Growth,Free Cash Growth
0,2005-12-31,0.085,0.131832,NaN,0.188475,0.154786
1,2010-12-31,0.436,0.128802,0.103752,0.128612,0.109920
2,2012-12-31,0.435,0.115804,0.081603,0.097979,0.138061
3,2015-12-31,0.228,0.049512,-0.120492,-0.019949,-0.185500
